# 모든 암호화폐의 일(Day) 캔들 가져오기

- 우리는 4, 5주차에서 암호화폐 데이터 분석 및 가격 예측 모델을 만들어 볼 것입니다   
- 이를 위해 각 암호화폐의 데이터가 필요하니 가져와보도록 합시다   
- [힌트]를 잘 참고하여 구현해주세요   

# 필요한 라이브러리 import 하기

In [37]:
import time
import requests
import pandas as pd
import json
from tqdm import tqdm

# API로 마켓 코드 조회하여 `market_code_df` 에 저장하기

In [25]:
url = "https://api.upbit.com/v1/market/all?isDetails=true"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

market_code_df = pd.DataFrame(json.loads(response.text))
market_code_df

,market_warning,market,korean_name,english_name
0,NONE,KRW-BTC,비트코인,Bitcoin
1,NONE,KRW-ETH,이더리움,Ethereum
2,NONE,BTC-ETH,이더리움,Ethereum
3,NONE,BTC-XRP,리플,Ripple
4,NONE,BTC-ETC,이더리움클래식,Ethereum Classic
...,...,...,...,...
299,NONE,BTC-ID,스페이스아이디,SPACE ID
300,NONE,BTC-AXL,엑셀라,Axelar
301,NONE,KRW-CTC,크레딧코인,Creditcoin
302,NONE,KRW-ASTR,아스타,Astar


# 원화로 거래 가능한 마켓 코드만 `market_code_df` 에남겨놓기

In [26]:
market_code_df = market_code_df[market_code_df["market"].str.contains("KRW")]
market_code_df

,market_warning,market,korean_name,english_name
0,NONE,KRW-BTC,비트코인,Bitcoin
1,NONE,KRW-ETH,이더리움,Ethereum
28,NONE,KRW-NEO,네오,NEO
29,NONE,KRW-MTL,메탈,Metal
30,NONE,KRW-XRP,리플,Ripple
...,...,...,...,...
292,NONE,KRW-IMX,이뮤터블엑스,Immutable X
295,NONE,KRW-SEI,세이,Sei
298,NONE,KRW-MINA,미나,Mina
301,NONE,KRW-CTC,크레딧코인,Creditcoin


# to에 넣을 과거 시각들을 담은 list 형태의 datetimes 변수 선언하기

In [27]:
datetimes = []
for year in range(2018,2025):
    datetimes.append(f"{year}-01-01T00%3A00%3A00%2B09%3A00")
    datetimes.append(f"{year}-07-01T00%3A00%3A00%2B09%3A00")
datetimes


['2018-01-01T00%3A00%3A00%2B09%3A00',
 '2018-07-01T00%3A00%3A00%2B09%3A00',
 '2019-01-01T00%3A00%3A00%2B09%3A00',
 '2019-07-01T00%3A00%3A00%2B09%3A00',
 '2020-01-01T00%3A00%3A00%2B09%3A00',
 '2020-07-01T00%3A00%3A00%2B09%3A00',
 '2021-01-01T00%3A00%3A00%2B09%3A00',
 '2021-07-01T00%3A00%3A00%2B09%3A00',
 '2022-01-01T00%3A00%3A00%2B09%3A00',
 '2022-07-01T00%3A00%3A00%2B09%3A00',
 '2023-01-01T00%3A00%3A00%2B09%3A00',
 '2023-07-01T00%3A00%3A00%2B09%3A00',
 '2024-01-01T00%3A00%3A00%2B09%3A00',
 '2024-07-01T00%3A00%3A00%2B09%3A00']

# 반년 단위로 캔들 가져와서 list 형태의 `dfs` 변수에 계속 추가해주기

In [42]:
headers = {"accept": "application/json"}
dfs = []
for datetime in datetimes:
    url = f"https://api.upbit.com/v1/candles/days?market=KRW-BTC&to={datetime}&count=200"
    response = requests.get(url, headers=headers)
    dfs.append(pd.DataFrame(json.loads(response.text)))
    time.sleep(0.04)

In [43]:
df = pd.concat(dfs)

In [30]:
df = df.drop(columns=["candle_date_time_utc", "timestamp", "prev_closing_price", "change_price"])
df.columns = [
    "market",
    "datetime",
    "open",
    "high",
    "low",
    "close",
    "trade_price",
    "trade_volume",
    "change_rate"
]

In [31]:
df = df.sort_values(by="datetime").reset_index(drop=True)
df = df.drop_duplicates()
df

,market,datetime,open,high,low,close,trade_price,trade_volume,change_rate
0,KRW-BTC,2017-09-25T09:00:00,4201000.0,4333000.0,4175000.0,4322000.0,5.602146e+08,132.484755,0.028803
1,KRW-BTC,2017-09-26T09:00:00,4317000.0,4418000.0,4311000.0,4321000.0,9.950724e+07,22.788340,-0.000231
2,KRW-BTC,2017-09-27T09:00:00,4322000.0,4677000.0,4318000.0,4657000.0,1.448276e+08,32.269662,0.077760
3,KRW-BTC,2017-09-28T09:00:00,4657000.0,4772000.0,4519000.0,4586000.0,3.721860e+08,80.588243,-0.015246
4,KRW-BTC,2017-09-29T09:00:00,4586000.0,4709000.0,4476000.0,4657000.0,2.724558e+08,59.352373,0.015482
...,...,...,...,...,...,...,...,...,...
2693,KRW-BTC,2024-01-17T09:00:00,59218000.0,59300000.0,58269000.0,58776000.0,1.666847e+11,2837.334490,-0.007095
2694,KRW-BTC,2024-01-18T09:00:00,58776000.0,58900000.0,56754000.0,57284000.0,2.472746e+11,4268.883840,-0.025385
2695,KRW-BTC,2024-01-19T09:00:00,57281000.0,58180000.0,55935000.0,57739000.0,3.199353e+11,5634.881771,0.007943
2696,KRW-BTC,2024-01-20T09:00:00,57738000.0,57796000.0,57242000.0,57517000.0,1.224661e+11,2127.464542,-0.003845


# 원화로 거래 가능한 암호화폐들을 각각의 csv로 저장해보아요

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
def get_coin(market_code, datetimes):
    headers = {"accept": "application/json"}
    dfs = []
    for datetime in datetimes:
        url = f"https://api.upbit.com/v1/candles/days?market={market_code}&to={datetime}&count=190"
        response = requests.get(url, headers=headers)
        dfs.append(pd.DataFrame(json.loads(response.text)))
        time.sleep(0.034)

    df = pd.concat(dfs)

    df = df.drop(columns=["candle_date_time_utc", "timestamp", "prev_closing_price", "change_price"])
    df.columns = [
        "market",
        "datetime",
        "open",
        "high",
        "low",
        "close",
        "trade_price",
        "trade_volume",
        "change_rate"
    ]

    df = df.sort_values(by="datetime").reset_index(drop=True)
    df = df.drop_duplicates()
    return df


In [47]:
base_path = "/content/drive/MyDrive/coin_data/days"

for market_code in tqdm(market_code_df["market"]):
    df = get_coin(market_code, datetimes)
    if len(df) >= 730:
        df.to_csv(base_path + f"/{market_code}.csv", index=False)

100%|██████████| 118/118 [22:25<00:00, 11.40s/it]


# days에 몇개의 암호화폐가 저장됐는지 살펴보기

In [41]:
import os
len(os.listdir(base_path))

101

- KRW로 구매 가능한 암호화폐는 총 118개였으나 저장된 파일은 101개인 것을 확인할 수 있습니다